In [3]:
import numpy as np
import tensorflow as tf

In [4]:
def shortcut(x,num_filter):
    out=tf.layers.conv2d(x,num_filter,3,use_bias=False)
    out=tf.layers.batch_normalization(out)
    out=tf.nn.relu(out)
    out=tf.layers.conv2d(out,num_filter,3)
    out=tf.layers.batch_normalization(out)
    out=out+x
    out=tf.nn.relu(out)
    return out

In [5]:
def transition(x,num_filter):
    out=tf.layers.conv2d(x,num_filter,3,strides=2,use_bias=False)
    out=tf.layers.batch_normalization(out)
    out=tf.nn.relu(out)
    out=tf.layers.conv2d(out,num_filter,3)
    out=tf.layers.batch_normalization(out)
    x=tf.layers.conv2d(x,num_filter,3,strides=2,use_bias=False)
    out=out+x
    out=tf.nn.relu(out)
    return out

In [6]:
def shortcutxn(out,num_filter,n):
    for i in range(n):
        out=shortcut(out,num_filter)
    return out

In [9]:
def resnet(x_dict,n_classes):
    x=x_dict['images']
    out=tf.layers.conv2d(x,64,7,strides=2,use_bias=False)
    out=tf.layers.batch_normalization(out)
    out=tf.nn.relu(out)
    out=tf.layers.MaxPooling2D(pool_size=2,strides=2)
    out=shortcutxn(out,64,3)
    out=transition(out,128)
    out=shortcutxn(out,128,3)
    out=transition(out,256)
    out=shortcutxn(out,256,5)
    out=transition(out,512)
    out=shortcutxn(out,512,2)
    out=tf.layers.AveragePooling2D(pool_size=7,strides=1)
    out=tf.layers.Flatten()(out)
    out=tf.layers.dense(out,10)
    return out

In [10]:
def model_fn(features,labels,mode):
    logits_train = resnet(features, num_classes)
    logits_test = resnet(features, num_classes)


    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)


    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)


    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())


    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)


    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [ ]:
from keras.datasets import cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
import cv2
x=[]
for i in range(len(x_test)):
    x.append(cv2.resize(x_test[i],(224,224)))
xte=np.array(x)
xte=xte/255.
xtr=[]
for i in range(len(x_train)):
    xtr.append(cv2.resize(x_train[i],(224,224)))
xtr=np.array(xtr)
xtr=xtr/255.

In [ ]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 2*60,  # Save checkpoints every 20 minutes.
    keep_checkpoint_max = 10       # Retain the 10 most recent checkpoints.
)
model = tf.estimator.Estimator(model_fn,model_dir='estimatormodel',config=my_checkpointing_config)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': xtr}, y=y_train,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

In [ ]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': xte}, y=y_test,
    batch_size=32, shuffle=False)
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])